In [1]:
import pandas as pd
import numpy as np
import gensim
from textblob import TextBlob
import re
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import PorterStemmer as ps
from gensim import corpora
from gensim import corpora, models, similarities

path = '/Users/deveshkandpal/Code/ADS/final-project/dataset/dataset/Step6/sentiment-nv/nv_sentiment.json'
nv_df = pd.read_json(path, orient='records', lines=True)


In [2]:
rest_review_dict = dict()
for temp in nv_df.iterrows() :
    row = temp[1]
    business_id = row.unique_business_id
    
    review_text = row['review_text']
    text = ' '.join([word for word in review_text.lower().split() if word not in STOPWORDS])
    
    review_stars = row['stars']
    if business_id in rest_review_dict :
        reviews_array = rest_review_dict[business_id]
        reviews_array.append({'review_text' : review_text, 'review_stars' : review_stars,
                              'polarity' : TextBlob(text).sentiment.polarity,
                             'stemmed_text' : text})
    else :
        reviews_array = list()
        reviews_array.append({'review_text' : review_text, 'review_stars' : review_stars,
                              'polarity' : TextBlob(text).sentiment.polarity,
                             'stemmed_text' : text})
        rest_review_dict[business_id] = reviews_array
    
        
        
    

In [3]:
business_corpus = dict()
for business_id, review_array in rest_review_dict.items():
    consolidated_text = [review['stemmed_text'] for review in review_array]
    texts = []
    for t in consolidated_text :
        for word in t.split(" ") :
            texts.append(word)
    texts = [texts]
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=2)
    topics = dict()
    for topic in lda.top_topics(corpus) :
        b = topic[0][0:15]
        for tup in b :
            if tup[1] not in topics :
                topics[tup[1]] = tup[0]
            else :
                if topics[tup[1]] < tup[0] :
                    topics[tup[1]] = tup[0]
    
    
    business_corpus[business_id] = topics

        
    


In [4]:
import pickle
pickle_out = open("nv_sentiment_corpus.json","wb")
pickle.dump(business_corpus, pickle_out)
pickle_out.close()

pickle_out = open("nv_bus_review_dict.json","wb")
pickle.dump(rest_review_dict, pickle_out)
pickle_out.close()